# LinkedIn Tango Puzzle
## What is Tango Puzzle
Tango puzzle is game that you need to guess place Moon or Sun in each cell of the puzzle according to certain rules.

![Tango Puzzle Example](./images/TangoPuzzle.png)

## How to Play Tango?
You will get a board with certain cells already filled out. you cannot change the value for those cells.
When filling the rest of the board, these are the rulles that you need to follow:
- atmost 2 sun or moon can eb horizontally or vertically next to each other.
- Each row and column must contain the same number of moons and suns.
- Some cells are separated by `=` sign. These two cells must have the same value.
- Some cells are separated by `x` sign. These cells must have an opposite value.

## Importing packages and setting constants

In [1]:
from typing import Optional
from dataclasses import dataclass

from ortools.sat.python.cp_model import CpModel, CpSolver, IntVar, OPTIMAL
from tabulate import tabulate

MOON: str = "\U0001F319"
SUN: str = "\U0001F31E"

## A Python dataclass to manage each cell of the puzzle.

In [2]:
@dataclass
class Cell:
    row_idx: int
    col_idx: int

    def is_horizontally_adjecent_to(self, cell: "Cell") -> bool:
        return (
            (self.row_idx == cell.row_idx) and 
            (abs(self.col_idx -  cell.col_idx) == 1)
        )

    def is_vertically_adjecent_to(self, cell: "Cell") -> bool:
        return (
            (self.col_idx == cell.col_idx) and 
            (abs(self.row_idx -  cell.row_idx) == 1)
        )

    def is_adjacent_to(self, cell: "Cell") -> bool:
        return self.is_horizontally_adjecent_to(cell) or self.is_vertically_adjecent_to(cell)

    @staticmethod
    def order(cell_1: "Cell", cell_2: "Cell") -> tuple["Cell", "Cell"]:
        if cell_1.is_horizontally_adjecent_to(cell_2):
            return (
                (cell_1, cell_2)
                if cell_1.col_idx < cell_2.col_idx else
                (cell_2, cell_1)
            )
        elif cell_1.is_vertically_adjecent_to(cell_2):
            return (
                (cell_1, cell_2)
                if cell_1.row_idx < cell_2.row_idx else
                (cell_2, cell_1)
            )
        else:
            raise ValueError("Cells must be adjacent, to be ordered.")

## Tango Solver

In [3]:
class TangoSolver:
    def __init__(
        self, 
        n_columns: int, 
        n_rows: int, 
        opposite_value_cells: list[tuple[Cell, Cell]],
        same_value_cells: list[tuple[Cell, Cell]],
        known_value_cells: list[tuple[Cell, int]],
    ):
        if n_columns % 2 != 0:
            raise ValueError(f"Number of columns ({n_columns}) must be an even number.")

        if n_rows % 2 != 0:
            raise ValueError(f"Number of rows ({n_rows}) must be an even number.")

        self._n_columns: int = n_columns
        self._n_rows: int = n_rows

        self._model_variables: list[list[IntVar]] = []

        self._model: Optional[CpModel] = None
        self._solver: Optionl[CpSolver] = None

        self._opposite_value_cells: list[tuple[Cell, Cell]] = []
        self._same_value_cells: list[tuple[Cell, Cell]] = []
        self._known_value_cells: list[tuple[Cell, int]] = []
        
        self.opposite_value_cells = opposite_value_cells
        self.same_value_cells = same_value_cells
        self.known_value_cells = known_value_cells

    @property
    def n_columns(self) -> int:
        return self._n_columns

    @property
    def n_rows(self) -> int:
        return self._n_rows

    @property
    def opposite_value_cells(self) -> list[tuple[Cell, Cell]]:
        return self._opposite_value_cells

    @opposite_value_cells.setter
    def opposite_value_cells(self, values: list[tuple[Cell, Cell]]):
        self._opposite_value_cells = [
            Cell.order(
                self.is_valid_cell(cell_1),
                self.is_valid_cell(cell_2)
            )
            for cell_1, cell_2 in values
        ]

    @property
    def same_value_cells(self) -> list[tuple[Cell, Cell]]:
        return self._same_value_cells

    @same_value_cells.setter
    def same_value_cells(self, values: list[tuple[Cell, Cell]]):
        self._same_value_cells = [
            Cell.order(
                self.is_valid_cell(cell_1),
                self.is_valid_cell(cell_2)
            )
            for cell_1, cell_2 in values
        ]
        
    @property
    def known_value_cells(self) -> list[Cell]:
        return self._known_value_cells

    @known_value_cells.setter
    def known_value_cells(self, cell_value_tuple: list[tuple[Cell, int]]):
        self._known_value_cells = [
            (self.is_valid_cell(cell), self.is_valid_value(value))
            for cell, value in cell_value_tuple
        ]

    @property
    def model(self) -> CpModel:
        if self._model is None:
            self._model = CpModel()
        return self._model

    @property
    def solver(self) -> CpSolver:
        if self._solver is None:
            self._solver = CpSolver()
        return self._solver

    def is_valid_row_idx(self, row_idx: int) -> int:
        if (row_idx < 0) or (row_idx >= self.n_rows):
            raise ValueError(f"row_idx ({row_idx}) must be between 0 and {self.n_rows - 1}.")

        return row_idx

    def is_valid_col_idx(self, col_idx: int) -> int:
        if (col_idx < 0) or (col_idx >= self.n_columns):
            raise ValueError(f"col_idx ({col_idx}) must be between 0 and {self.n_columns - 1}.")

        return col_idx

    def is_valid_cell(self, cell: Cell) -> Cell:
        return Cell(
            self.is_valid_row_idx(cell.row_idx),
            self.is_valid_col_idx(cell.col_idx)
        )

    @staticmethod
    def is_valid_value(value: int) -> int:
        if value not in {0, 1}:
            raise ValueError(f"value ({value}) must be either 0 (moon) or sun(1).")

        return value

    def get_variable(self, row_idx: int, col_idx: int) -> IntVar:
        return self._model_variables[self.is_valid_row_idx(row_idx)][self.is_valid_col_idx(col_idx)]

    def get_value(self, row_idx: int, col_idx: int) -> int:
        return self.solver.value(self.get_variable(row_idx, col_idx))

    def get(self, cell: Cell):
        return self.get_variable(cell.row_idx, cell.col_idx)

    def get_cell_value(self, cell: Cell) -> int:
        return self.get_value(cell.row_idx, cell.col_idx)

    def _initialize_model(self) -> None:
        self._model = None
        self._solver = None
        self._model_variables = [
            [
                self.model.NewBoolVar(self.get_cell_name(row_idx, col_idx))
                for col_idx in range(self.n_columns)
            ]
            for row_idx in range(self.n_rows)    
        ]
        
        # no more than two consecutive Moon or Sun in a row
        for row_idx in range(self.n_rows):
            for col_idx in range(self.n_columns - 2):
                self.model.add(sum(self._model_variables[row_idx][col_idx:col_idx+3]) != 0)
                self.model.add(sum(self._model_variables[row_idx][col_idx:col_idx+3]) != 3)

        # no more than two consecutive moon or sun in a column
        for col_idx in range(self.n_columns):
            for row_idx in range(self.n_rows - 2):
                tmp_array = [
                    self.get_variable(row_idx, col_idx),
                    self.get_variable(row_idx + 1, col_idx),
                    self.get_variable(row_idx + 2, col_idx),
                ]
                self.model.add(sum(tmp_array) != 0)
                self.model.add(sum(tmp_array) != 3)

        # Same Number of Sun and Moon in each row:
        for row_idx in range(self.n_rows):
                self.model.add(sum(self._model_variables[row_idx][:]) == (self.n_columns//2))

        # Same Number of Sun and Moon in each column;
        for col_idx in range(self.n_columns):
            self.model.add(
                sum([
                    self._model_variables[row_idx][col_idx]
                    for row_idx in range(self.n_rows)
                ]) == (self.n_rows//2)
            )
                

    def add_opposite_values_constraint(self, cell_1: Cell, cell_2: Cell) -> "TangoSolver":
        if not cell_1.is_adjacent_to(cell_2):
            raise ValueError(f"The cells are not adjacents. cell_1: {cell_1} and cell_2: {cell_2}")
            
        self.model.add(self.get(cell_1) != self.get(cell_2))
        
        return self

    def add_same_values_constraints(self, cell_1: Cell, cell_2: Cell) -> "TangoSolver":
        if not cell_1.is_adjacent_to(cell_2):
            raise ValueError(f"The cells are not adjacents. cell_1: {cell_1} and cell_2: {cell_2}")
            
        self.model.add(self.get(cell_1) == self.get(cell_2))
        
        return self
    
    def add_known_values_constraint(self, cell: Cell, value: int) -> "TangoSolver":
        self.model.add(self.get(self.is_valid_cell(cell)) == self.is_valid_value(value))

        return self

    @staticmethod
    def get_cell_name(row_idx: int, col_idx: int) -> str:
        return f"c_{row_idx}_{col_idx}"

    def solve(self, print_solution: bool = False) -> bool:
        self._initialize_model()

        # enforce opposite value cells
        for cell_1, cell_2 in self.opposite_value_cells:
            self.add_opposite_values_constraint(cell_1, cell_2)

        # enforce same value cells
        for cell_1, cell_2 in self.same_value_cells:
            self.add_same_values_constraints(cell_1, cell_2)

        # enforce known value cells
        for cell, value in self.known_value_cells:
            self.add_known_values_constraint(cell, value)
            
        status = self.solver.solve(self.model)

        output = status == OPTIMAL
        if output and print_solution:
            solution = [
                [
                    SUN if self.get_value(row_idx, col_idx) == 1 else MOON
                    for col_idx in range(6)
                ]
                for row_idx in range(6)    
            ]

            print(tabulate(solution, tablefmt="grid"))

        return output


## Trying it out

### 2025-11-14

In [4]:
tango_solver = TangoSolver(
    n_columns=6, 
    n_rows=6, 
    opposite_value_cells=[
        (Cell(0, 1), Cell(0, 2)),
        (Cell(0, 3), Cell(0, 4)),
        (Cell(1, 0), Cell(2, 0)),
        (Cell(3, 0), Cell(4, 0)),
        (Cell(1, 5), Cell(2, 5)),
        (Cell(5, 1), Cell(5, 2)),
    ],
    same_value_cells=[
        (Cell(5, 3), Cell(5, 4)),
        (Cell(3, 5), Cell(4, 5)),
    ],
    known_value_cells=[
        (Cell(0, 0), 0),
        (Cell(0, 5), 1),
        (Cell(1, 1), 0),
        (Cell(1, 4), 0),
        (Cell(4, 1), 0),
        (Cell(4, 5), 0),
        (Cell(5, 0), 1),
        (Cell(5, 5), 1),
    ],
)

if tango_solver.solve(print_solution=True):
    print("Found a solution.")
else:
    print("Failed to find a solution.")


+----+----+----+----+----+----+
| 🌙 | 🌞 | 🌙 | 🌙 | 🌞 | 🌞 |
+----+----+----+----+----+----+
| 🌞 | 🌙 | 🌞 | 🌞 | 🌙 | 🌙 |
+----+----+----+----+----+----+
| 🌙 | 🌞 | 🌙 | 🌙 | 🌞 | 🌞 |
+----+----+----+----+----+----+
| 🌙 | 🌞 | 🌙 | 🌞 | 🌞 | 🌙 |
+----+----+----+----+----+----+
| 🌞 | 🌙 | 🌞 | 🌞 | 🌙 | 🌙 |
+----+----+----+----+----+----+
| 🌞 | 🌙 | 🌞 | 🌙 | 🌙 | 🌞 |
+----+----+----+----+----+----+
Found a solution.


### 2025-11-15

In [5]:
tango_solver = TangoSolver(
    n_columns=6, 
    n_rows=6, 
    opposite_value_cells=[
    ],
    same_value_cells=[
        (Cell(0, 3), Cell(0, 4)),
        (Cell(1, 2), Cell(1, 3)),
        (Cell(0, 3), Cell(1, 3)),
    ],
    known_value_cells=[
        (Cell(2, 0), 0),
        (Cell(3, 0), 1),
        (Cell(4, 0), 0),
        (Cell(2, 4), 1),
        (Cell(3, 4), 0),
        (Cell(4, 4), 0),
        (Cell(3, 5), 1),
        (Cell(5, 1), 1),
        (Cell(5, 2), 0),
        (Cell(5, 3), 1),
    ],
)

if tango_solver.solve(print_solution=True):
    print("Found a solution.")
else:
    print("Failed to find a solution.")

+----+----+----+----+----+----+
| 🌞 | 🌙 | 🌞 | 🌙 | 🌙 | 🌞 |
+----+----+----+----+----+----+
| 🌞 | 🌞 | 🌙 | 🌙 | 🌞 | 🌙 |
+----+----+----+----+----+----+
| 🌙 | 🌞 | 🌙 | 🌞 | 🌞 | 🌙 |
+----+----+----+----+----+----+
| 🌞 | 🌙 | 🌞 | 🌙 | 🌙 | 🌞 |
+----+----+----+----+----+----+
| 🌙 | 🌙 | 🌞 | 🌞 | 🌙 | 🌞 |
+----+----+----+----+----+----+
| 🌙 | 🌞 | 🌙 | 🌞 | 🌞 | 🌙 |
+----+----+----+----+----+----+
Found a solution.
